In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [2]:
ua = UserAgent()
headers = {"User-Agent": ua.random}

url = 'https://news.naver.com/main/ranking/popularDay.naver'
res = requests.get(url, headers=headers)
res.status_code

200

In [3]:
soup = BeautifulSoup(res.content, "html.parser")
# print(soup.prettify())

1. 언론사별 박스 선택
    > select 메서드로 여러 개 선택
2. 각 박스 안의 링크들을 모두 가져옴
    > 하나의 박스를 선택했을 때 그 안의 링크들을 추출
3. 링크를 get 요청해서 html 문자열을 가져옴
4. 뉴스의 제목과 본문을 추출하여 저장

- 언론사별 뉴스 박스 선택하기

In [4]:
news_box_list = soup.select("._officeCard0 > .rankingnews_box")
len(news_box_list)

12

In [5]:
news_box_list[5].select_one('.rankingnews_name').text # 언론사명

'경향신문'

In [6]:
news_list = news_box_list[0].select('.rankingnews_list .list_title') # 언론사별 뉴스 링크
news_list

[<a class="list_title nclicks('RBP.rnknws')" href="https://n.news.naver.com/article/214/0001376490?ntype=RANKING">NBS 尹 지지율도 '역대 최저'‥'김건희 특검법 찬성' 65%</a>,
 <a class="list_title nclicks('RBP.rnknws')" href="https://n.news.naver.com/article/214/0001376485?ntype=RANKING">"명태균, 김 여사 통화 들려줬다"‥계속되는 폭로 "국감 나갈 것"</a>,
 <a class="list_title nclicks('RBP.rnknws')" href="https://n.news.naver.com/article/214/0001376475?ntype=RANKING">"무속인이 윤 대통령 서거 얘기"‥인요한 의원실 전화한 남성 경찰 조사</a>,
 <a class="list_title nclicks('RBP.rnknws')" href="https://n.news.naver.com/article/214/0001376449?ntype=RANKING">"김태효, 국기에 대한 경례 거부"‥폭발한 野 "파면 요구"</a>,
 <a class="list_title nclicks('RBP.rnknws')" href="https://n.news.naver.com/article/214/0001376466?ntype=RANKING">"맹장 수술도 못 한다고요?" 부산 헤맨 중학생 어머니</a>]

In [17]:
for a_tag in news_list:
    link = a_tag.get('href')
    print(link)
    # break

https://n.news.naver.com/article/020/0003589226?ntype=RANKING
https://n.news.naver.com/article/020/0003589298?ntype=RANKING
https://n.news.naver.com/article/020/0003589292?ntype=RANKING
https://n.news.naver.com/article/020/0003589303?ntype=RANKING
https://n.news.naver.com/article/020/0003589297?ntype=RANKING


- 뉴스 본문 보기

In [8]:
res_news = requests.get(link, headers=headers)
res_news.status_code

200

In [9]:
soup_news = BeautifulSoup(res_news.content, "html.parser")
# soup_news

In [18]:
soup_news.select_one("#title_area span").text # 뉴스 제목

'“구급대원 연기해 보니 노고와 희생 느껴져”…5000만 원 기부한 배우'

In [21]:
soup_news.select_one("#newsct_article").text # 뉴스 본문

'\n\n\n\n\n\n사진출처=tvN배우 김지은이 순직 소방공무원과 유가족들을 위해 5000만 원을 기부했다.김지은 소속사 HB엔터테인먼트는 “김지은이 국민의 안전과 생명을 지키는 소방공무원의 헌신에 보답하는 마음으로 23일 사단법인 소방가족희망나눔에 5000만 원의 후원금을 기부했다”고 밝혔다.현재 방영 중인 tvN ‘엄마친구아들’에서 구급대원 정모음 역으로 출연 중인 김지은은 “이번 드라마를 통해 소방공무원 분들의 노고와 희생을 몸소 느끼게 됐다”며 “어떤 방법으로 감사함과 존경심을 표할 수 있을까 고민하던 중 기부를 결정했다”고 전했다.\n\n\n\n배우 김지은이 대한소방공제회에서 5000만 원을 소방가족희망나눔에 기부했다. HB엔터테인먼트 제공. 김지은이 전한 후원금은 순직 소방공무원의 유가족 예우 지원뿐만 아니라 소방청에서 주최하는 ‘제1회 119메모리얼데이’ 행사를 위해서 사용될 예정이다. ‘119 메모리얼대회’는 순직 소방공무원에 대한 대국민 예우 문화를 조성하는 국민 참여형 추모행사다.\n\n'

```python
[
    {"언론사명": "", "뉴스제목": "", "뉴스본문": ""},
    {"언론사명": "", "뉴스제목": "", "뉴스본문": ""},
    {"언론사명": "", "뉴스제목": "", "뉴스본문": ""},
    ...
]
```

In [12]:
ua = UserAgent()

while True:
    headers = {"User-Agent": ua.random}
    url = 'https://news.naver.com/main/ranking/popularDay.naver'
    res = requests.get(url, headers=headers)

    soup = BeautifulSoup(res.content, 'html.parser')
    news_box_list = soup.select("._officeCard0 > .rankingnews_box")
    if len(news_box_list) > 0:
        break

news_data = [] # 모든 뉴스들을 담을 리스트

for news_box in news_box_list:
    news_list = news_box.select('.rankingnews_list .list_title') # 언론사별 뉴스 링크
    
    for a_tag in news_list:
        news_dict = {} # 하나의 뉴스가 들어있는 딕셔너리
        news_dict['언론사명'] = news_box.select_one('.rankingnews_name').text # 언론사명
        
        try:
            res_news = requests.get(a_tag['href'], headers=headers)
        except Exception as e:
            print(e)
        else:    
            soup_news = BeautifulSoup(res_news.content, 'html.parser')
            news_dict['뉴스제목'] = soup_news.select_one("#title_area span").text # 뉴스 제목
            news_dict['뉴스본문'] = soup_news.select_one("#newsct_article").text # 뉴스 본문
            news_data.append(news_dict)

In [13]:
len(news_data)

60

In [ ]:
import pandas as pd
pd.DataFrame(news_data)